# Linear System Model Reduction

In [ ]:
%pylab inline
import numpy as np
import pylab
try:
    import seaborn as sns  # optional; prettier graphs
except ImportError:
    pass

from scipy.linalg import solve_lyapunov, cholesky, svd, inv
from nengo.utils.numpy import rmse

import nengolib

### Problem Statement

Suppose we have some linear system. We'd like to reduce its order while maintaining as much of its characteristics as possible. Take, for example, a lowpass filter that has a small amount of 3rd-order dynamics mixed in, resulting in a 4th-order system that consists mostly of first-order dynamics.

In [ ]:
isys = nengolib.Lowpass(0.05)
noise = 0.5*nengolib.Lowpass(0.01) + 0.5*nengolib.Alpha(0.005)
p = 0.8
sys = p*isys + (1-p)*noise

### Exact Minimal Realizations

By cancelling repeated zeros and poles from a system, we can obtain an exact version of that same system with potentially reduced order. However, for the above system, there are no poles to be cancelled, and so this does not help to reduce the order. 

In [ ]:
assert nengolib.signal.minreal(isys/isys) == nengolib.signal.LinearSystem(1)

minsys = nengolib.signal.minreal(sys)
assert minsys == sys

As a crude way of getting around this problem, we can raise the tolerance for detecting similar poles/zeros until repeats are found. By setting the tolerance appropriately for this example, we can reduce the model to a first-order filter, with a surprisingly similar response. However, as we will soon see further down, we can do even better.

In [ ]:
minsys_crude = nengolib.signal.minreal(sys, tol=300.0)
assert minsys_crude.order_den == 1

def test_sys(u, redsys, dt=0.001):
    orig = nengolib.signal.apply_filter(u, sys, dt=dt)
    red = nengolib.signal.apply_filter(u, redsys, dt=dt)
    
    pylab.figure()
    pylab.title("(RMSE: %s)" % rmse(orig, red))
    pylab.plot(orig, label="Original")
    pylab.plot(red, label="Reduced")
    pylab.legend()
    pylab.show()

rng = np.random.RandomState(0)
white = rng.normal(size=100)
test_sys(white, minsys_crude)

### Balanced Realizations

First we need to compute some special matrices from Lyapunov theory.

The "controllability gramian" (a.k.a. "reachability gramian" for linear systems) $P$ satisfies: 

$$AP + PA' = -BB'$$

The "observability gramian" $Q$ satisfies:

$$A'Q + QA = -C'C$$

See [2] for more background information.

In [ ]:
A, B, C, D = sys.ss

P = nengolib.signal.control_gram(sys)
assert np.allclose(np.dot(A, P) + np.dot(P, A.T), -np.dot(B, B.T))

Q = nengolib.signal.observe_gram(sys)
assert np.allclose(np.dot(A.T, Q) + np.dot(Q, A), -np.dot(C.T, C))

The algorithm from [3] computes the lower cholesky factorization of $P \, ( = RR')$, and the singular value decomposition of $R'QR$.

In [ ]:
R = cholesky(P, lower=True)
assert np.allclose(P, np.dot(R, R.T))

M = np.dot(np.dot(R.T, Q), R)
U, S, V = svd(M)  # note: squared

T = np.dot(R, U) * S ** (-1. / 2)

This gives the similarity transform [4] that maps the state to the "balanced realization" of the same order. This system is equivalent up to a change of basis $T$ in the state-space.

In [ ]:
TA, TB, TC, TD = nengolib.signal.similarity_transform(A, B, C, D, T)
assert nengolib.signal.sys_equal(sys, (TA, TB, TC, TD))

And the reason we do this is because the singular values reflect a measure of importance for each of the states in the new realization. The order should then be reduced by removing the least important states.

In [ ]:
pylab.figure()
pylab.scatter(range(len(S)), S)
pylab.show()

The short-cut to do the above procedure in `nengolib` is the function `balreal`:

In [ ]:
sys_check, S_check = nengolib.signal.balreal(sys)

assert np.allclose(TA, sys_check.ss[0])
assert np.allclose(TB, sys_check.ss[1])
assert np.allclose(TC, sys_check.ss[2])
assert np.allclose(TD, sys_check.ss[3])
assert np.allclose(S, S_check)

### Approximate Model Order Reduction

Low singular values indicate states are less important. The method in [5] can be used to remove these states while matching the DC gain for continuous or discrete systems.

In [ ]:
redsys = nengolib.signal.modred((TA, TB, TC, TD), 0, method='dc')
assert redsys.order_den == 1

In [ ]:
step = np.zeros(300)
step[50:] = 1.0
test_sys(step, redsys)
test_sys(white, redsys)

However, this doesn't work very well for matching the response of the system given white-noise input. If we care less about the steady-state response, then it is much more accurate to simply delete the less important states.

In [ ]:
delsys = nengolib.signal.modred((TA, TB, TC, TD), 0, method='del')
test_sys(white, delsys)

The combination of `balreal` and `modred` can be executed together by calling `nengolib.signal.balred` with a desired order and method.

### References

[1] http://www.mathworks.com/help/control/ref/minreal.html

[2] https://en.wikibooks.org/wiki/Control_Systems/Controllability_and_Observability

[3] https://people.kth.se/~hsan/modred_files/L4.pdf

[4] http://www.mathworks.com/help/control/ref/balreal.html

[5] http://www.mathworks.com/help/control/ref/modred.html